# 🖼️ imagefeatures Demo

Extract classic image features using Python with minimal dependencies.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kelkalot/imagefeatures/blob/main/examples/demo.ipynb)

## 1. Installation

In [ ]:
# Install from GitHub
!pip install -q git+https://github.com/kelkalot/imagefeatures.git

## 2. Download Sample Images

In [ ]:
import urllib.request
import os

# Create sample images directory
os.makedirs('sample_images', exist_ok=True)

# Download sample images
sample_urls = [
    ('https://picsum.photos/400/300?random=1', 'sample_images/image1.jpg'),
    ('https://picsum.photos/400/300?random=2', 'sample_images/image2.jpg'),
    ('https://picsum.photos/400/300?random=3', 'sample_images/image3.jpg'),
]

for url, path in sample_urls:
    if not os.path.exists(path):
        urllib.request.urlretrieve(url, path)
        print(f'Downloaded: {path}')

print('\nSample images ready!')

## 3. Extract Features from a Single Image

In [ ]:
from imagefeatures import FeatureExtractor
from imagefeatures.features import ColorHistogram, LocalBinaryPatterns, CEDD

# Create extractor with selected features
extractor = FeatureExtractor([
    ColorHistogram(),
    LocalBinaryPatterns(),
    CEDD()
])

# Extract from single image
features = extractor.extract('sample_images/image1.jpg')

# Display results
print('Extracted features:')
for name, vector in features.items():
    print(f'  {name}: {vector.shape[0]} dimensions, range [{vector.min():.2f}, {vector.max():.2f}]')

## 4. Extract All Features from a Folder

In [ ]:
from imagefeatures import FeatureExtractor

# Use all available features (22 total, 3058 dimensions)
extractor = FeatureExtractor()

# Extract from folder
result = extractor.extract_folder('sample_images/', output='features.csv')

print(f'Processed {len(result["filenames"])} images')
print(f'Total features: {len(result["feature_names"])}')
print(f'Total dimensions: {result["features"].shape[1]}')
print(f'\nFeatures extracted:')
for name in result['feature_names']:
    print(f'  - {name}')

## 5. Use with scikit-learn

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity between images
X = result['features']
filenames = result['filenames']

# Cosine similarity matrix
similarity = cosine_similarity(X)

print('Image Similarity Matrix:')
print('\n' + ' ' * 15, end='')
for name in filenames:
    print(f'{os.path.basename(name)[:10]:>12}', end='')
print()

for i, name in enumerate(filenames):
    print(f'{os.path.basename(name)[:15]:15}', end='')
    for j in range(len(filenames)):
        print(f'{similarity[i,j]:12.3f}', end='')
    print()

## 6. Visualize Feature Vectors

In [ ]:
import matplotlib.pyplot as plt
from imagefeatures.features import ColorHistogram, LocalBinaryPatterns
from imagefeatures.utils import load_image

# Load image
img = load_image('sample_images/image1.jpg')

# Extract specific features
color_hist = ColorHistogram()
color_hist.extract(img)

lbp = LocalBinaryPatterns()
lbp.extract(img)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Original image
axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[0].axis('off')

# Color histogram
axes[1].bar(range(64), color_hist.get_feature_vector(), color='steelblue')
axes[1].set_title('Color Histogram (64 bins)')
axes[1].set_xlabel('Bin')
axes[1].set_ylabel('Value')

# LBP histogram
axes[2].bar(range(256), lbp.get_feature_vector(), width=1, color='coral')
axes[2].set_title('LBP Histogram (256 bins)')
axes[2].set_xlabel('LBP Code')
axes[2].set_ylabel('Value')

plt.tight_layout()
plt.show()

## 7. Custom Feature Selection

In [ ]:
from imagefeatures import FeatureExtractor
from imagefeatures.features import (
    CEDD, FCTH, JCD,            # Combined color+texture
    Gabor, Tamura, Haralick,    # Texture features
    ColorLayout, HuMoments      # Shape/layout features
)

# Create custom extractor with lightweight features
custom_extractor = FeatureExtractor([
    CEDD(),           # 144d
    Gabor(),          # 48d
    ColorLayout(),    # 12d
    HuMoments(),      # 7d
    Haralick(),       # 6d
])

print('Custom extractor features:')
for f in custom_extractor.features:
    print(f'  {f.name}: {f.dim} dimensions')

print(f'\nTotal dimensions: {custom_extractor.get_total_dimensions()}')

## 8. Image Retrieval Example

In [ ]:
import numpy as np
from imagefeatures import FeatureExtractor
from imagefeatures.features import CEDD, LocalBinaryPatterns, ColorMoments

# Build feature database with compact features
extractor = FeatureExtractor([CEDD(), LocalBinaryPatterns(), ColorMoments()])
db = extractor.extract_folder('sample_images/')

# Query with first image
query_idx = 0
query_features = db['features'][query_idx]
query_name = os.path.basename(db['filenames'][query_idx])

# Find most similar images
distances = []
for i, features in enumerate(db['features']):
    if i != query_idx:
        dist = np.linalg.norm(query_features - features)
        distances.append((os.path.basename(db['filenames'][i]), dist))

# Sort by distance
distances.sort(key=lambda x: x[1])

print(f'Query: {query_name}')
print('\nMost similar images:')
for name, dist in distances:
    print(f'  {name}: distance = {dist:.2f}')

## 9. Create Your Own Feature

In [ ]:
from imagefeatures import GlobalFeature, register_feature, FeatureExtractor
import numpy as np

@register_feature("mean_rgb")
class MeanRGB(GlobalFeature):
    """Simple feature: mean RGB values."""
    
    def __init__(self):
        self._vector = None
    
    def extract(self, image: np.ndarray) -> None:
        self._vector = np.mean(image, axis=(0, 1))
    
    def get_feature_vector(self) -> np.ndarray:
        return self._vector if self._vector is not None else np.zeros(3)
    
    @property
    def name(self) -> str:
        return "mean_rgb"

# Use custom feature
extractor = FeatureExtractor([MeanRGB()])
features = extractor.extract('sample_images/image1.jpg')
print(f'Mean RGB: {features["mean_rgb"]}')

---

## 📚 All Available Features (22 total)

| Category | Feature | Dims | Description |
|----------|---------|------|-------------|
| **Color** | ColorHistogram | 64 | RGB histogram |
| | ColorMoments | 9 | Mean, std, skewness |
| | OpponentHistogram | 512 | Opponent color space |
| | FuzzyColorHistogram | 72 | Fuzzy HSV |
| | DominantColors | 20 | K-means colors |
| | ScalableColor | 64 | MPEG-7 scalable |
| **Texture** | LocalBinaryPatterns | 256 | Classic LBP |
| | RotationInvariantLBP | 36 | Rotation-invariant |
| | Gabor | 48 | Gabor wavelets |
| | Tamura | 18 | Coarseness/contrast |
| | Haralick | 6 | GLCM features |
| | Centrist | 256 | Census transform |
| **Shape** | EdgeHistogram | 80 | MPEG-7 edges |
| | PHOG | 630 | Pyramid HOG |
| | HOG | 144 | Histogram of gradients |
| | HuMoments | 7 | Invariant moments |
| **Layout** | ColorLayout | 12 | MPEG-7 DCT |
| | LuminanceLayout | 64 | Luminance DCT |
| **Combined** | CEDD | 144 | Color + edge |
| | FCTH | 192 | Color + texture |
| | JCD | 168 | CEDD + FCTH |
| | AutoColorCorrelogram | 256 | Spatial correlation |

**Total: 3,058 dimensions**